<a href="https://colab.research.google.com/github/madhuselvaraj/tensorflow_projects/blob/master/titanic_with_convolutionnetwrk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we are goinng to deal with a dataset of the people who travelled in  titanic and predict which people we likely to survive in it.

import files.

In [0]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import io
import numpy as np

upload training dataset file 'train.csv' in google colab from local 

In [2]:
from google.colab import files
uploaded = files.upload()


Saving train.csv to train.csv


In [0]:
train_df = pd.read_csv(io.BytesIO(uploaded['train.csv']))

upload test dataset file 'test.csv' in google colab from local

In [4]:
from google.colab import files
uploaded = files.upload()


Saving test.csv to test.csv


In [0]:
test_df = pd.read_csv(io.BytesIO(uploaded['test.csv']))

check for the features we have in train and test datasets

In [6]:
print(train_df.columns.values)
print(test_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']
['PassengerId' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare'
 'Cabin' 'Embarked']


In [9]:
"before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

NameError: ignored

Droping Ticket' and 'Cabin' features as they are not exactly needed for training.

In [0]:
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]


In [11]:
"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

('After', (891, 10), (418, 9), (891, 10), (418, 9))

Adding a column called 'Title' through extracting the surname from the name column.
Calculating the 'Sex' column  for each row of 'Title'

In [12]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


Replacing the values of 'Title' column with  maximum repeated ones.

In [0]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'others')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

Grouping the "Title" and "Survived " column for calulating the preferences which were given.

In [14]:
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,others,0.347826


From this we can  see that women and children were given preferences than others.

replacing it with digits.

In [0]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)


In [16]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,3.0
1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2.0
2,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,3.0
3,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,11.1333,S,3.0
4,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,30.0708,C,3.0


Droping 'Name' and 'PassengerId' features as they are not exactly needed for training.

In [17]:
train_df = train_df.drop(['Name','PassengerId'],axis = 1)
test_df = test_df.drop(['Name'],axis = 1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 9), (418, 9))

In [18]:
train_df.tail()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
886,0,3,male,33.0,0,0,7.8958,S,1.0
887,0,2,male,28.0,0,0,10.5000,S,1.0
888,0,3,male,25.0,0,0,7.0500,S,1.0
889,0,2,male,27.0,0,0,13.0000,S,0.0
890,0,3,male,32.0,0,0,7.7500,Q,1.0


Replacing 'male' with '0' and 'female' with '1'.

In [0]:
sex_mapping ={'male':0,'female':1}
for dataset in combine:
  dataset['Sex'] = dataset['Sex'].map(sex_mapping).astype(int)

In [20]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,1,1,38.0,1,0,71.2833,C,3.0
1,1,3,1,26.0,0,0,7.9250,S,2.0
2,1,1,1,35.0,1,0,53.1000,S,3.0
3,1,3,1,27.0,0,2,11.1333,S,3.0
4,1,2,1,14.0,1,0,30.0708,C,3.0


Replacing null in 'Age' column with an average value.

In [21]:
guess_ages = np.zeros((2,3))
guess_ages

array([[0., 0., 0.],
       [0., 0., 0.]])

In [22]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,1,1,38,1,0,71.2833,C,3.0
1,1,3,1,26,0,0,7.9250,S,2.0
2,1,1,1,35,1,0,53.1000,S,3.0
3,1,3,1,27,0,2,11.1333,S,3.0
4,1,2,1,14,1,0,30.0708,C,3.0


Adding 'AgeBand' column.

In [23]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16.0]",0.550000
1,"(16.0, 32.0]",0.337374
2,"(32.0, 48.0]",0.412037
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


Replacing Age with 0,1,2,3 inorder to increase the training efficiency.

In [24]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,1,1,1,2,1,0,71.2833,C,3.0,"(32.0, 48.0]"
1,1,3,1,1,0,0,7.9250,S,2.0,"(16.0, 32.0]"
2,1,1,1,2,1,0,53.1000,S,3.0,"(32.0, 48.0]"
3,1,3,1,1,0,2,11.1333,S,3.0,"(16.0, 32.0]"
4,1,2,1,0,1,0,30.0708,C,3.0,"(-0.08, 16.0]"


Adding a column 'FamilySize' denoting size of family

In [25]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


we can analize the probability of survived if the person is alone through a column 'IsAlone' 

In [26]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()


,IsAlone,Survived
0,0,0.505650
1,1,0.303538


Droping 'Parch' , 'SibSp', 'FamilySize'  and 'AgeBand' features as they are not exactly needed for training.

In [27]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize','AgeBand'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,1,1,1,2,71.2833,C,3.0,0
1,1,3,1,1,7.9250,S,2.0,1
2,1,1,1,2,53.1000,S,3.0,0
3,1,3,1,1,11.1333,S,3.0,0
4,1,2,1,0,30.0708,C,3.0,0


Creating a new column through combining Age and Pclass.

In [0]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

check for frequent Embarked and replace it for null values.

In [29]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

'S'

In [30]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 2, 'C': 0, 'Q': 1} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,1,1,1,2,71.2833,0,3.0,0,2
1,1,3,1,1,7.9250,2,2.0,1,3
2,1,1,1,2,53.1000,2,3.0,0,2
3,1,3,1,1,11.1333,2,3.0,0,3
4,1,2,1,0,30.0708,0,3.0,0,0


Convert the fare values to single digit through median.

In [31]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,892,3,0,2,7.8292,1,1.0,1,6
1,893,3,1,2,7.0000,2,3.0,0,6
2,894,2,0,3,9.6875,1,1.0,1,6
3,895,3,0,1,8.6625,2,1.0,1,3
4,896,3,1,1,12.2875,2,3.0,0,3


Replace the values of "FareBand" with 0,1,2,3

In [32]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [33]:
for dataset in combine:    
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31.0), 'Fare'] = 2
    dataset.loc[(dataset['Fare'] > 31.0) & (dataset['Fare'] <= 512.329), 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
train_df.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,1,1,1,2,3,0,3.0,0,2
1,1,3,1,1,1,2,2.0,1,3
2,1,1,1,2,3,2,3.0,0,2
3,1,3,1,1,1,2,3.0,0,3
4,1,2,1,0,2,0,3.0,0,0
5,1,3,1,0,2,2,2.0,0,0
6,1,1,1,3,2,2,2.0,1,3
7,1,2,1,3,2,2,3.0,1,6
8,1,3,1,1,0,0,3.0,1,3
9,1,3,1,0,1,1,2.0,1,0


Droping 'Survived' and 'PassengerId' features as they are not exactly needed for training and testing

In [34]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [35]:
print(X_train.columns.values)

['Pclass' 'Sex' 'Age' 'Fare' 'Embarked' 'Title' 'IsAlone' 'Age*Class']


In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


define model

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(7, activation='relu',input_dim = 8),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 7)                 63        
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 24        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 4         
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.fit(X_train, Y_train,epochs = 100)

Epoch 1/100
891/891 [==============================] - 0s 257us/sample - loss: 0.6705 - acc: 0.6566
Epoch 2/100
891/891 [==============================] - 0s 41us/sample - loss: 0.6310 - acc: 0.6936
Epoch 3/100
891/891 [==============================] - 0s 41us/sample - loss: 0.5974 - acc: 0.7419
Epoch 4/100
891/891 [==============================] - 0s 43us/sample - loss: 0.5658 - acc: 0.7710
Epoch 5/100
891/891 [==============================] - 0s 42us/sample - loss: 0.5386 - acc: 0.7722
Epoch 6/100
891/891 [==============================] - 0s 41us/sample - loss: 0.5143 - acc: 0.7733
Epoch 7/100
891/891 [==============================] - 0s 41us/sample - loss: 0.4931 - acc: 0.7789
Epoch 8/100
891/891 [==============================] - 0s 41us/sample - loss: 0.4749 - acc: 0.7901
Epoch 9/100
891/891 [==============================] - 0s 43us/sample - loss: 0.4607 - acc: 0.8058
Epoch 10/100
891/891 [==============================] - 0s 44us/sample - loss: 0.4499 - acc: 0.8126
Epoch 11

In [53]:
Y_test = model.predict(X_test).tolist()
print(Y_test)

[[0.0758213996887207], [0.21558350324630737], [0.12258130311965942], [0.092805415391922], [0.36959606409072876], [0.12497848272323608], [0.5889694094657898], [0.10646942257881165], [0.7073131799697876], [0.0657162070274353], [0.09374341368675232], [0.284619003534317], [0.987958550453186], [0.0962144136428833], [0.9819850921630859], [0.9535790681838989], [0.14669093489646912], [0.11237630248069763], [0.20113897323608398], [0.44321176409721375], [0.29821163415908813], [0.46505776047706604], [0.986405611038208], [0.5246872901916504], [0.9906526803970337], [0.04601815342903137], [0.9909005761146545], [0.11237630248069763], [0.284619003534317], [0.08216837048530579], [0.0962144136428833], [0.10537689924240112], [0.21455487608909607], [0.3628416955471039], [0.5246872901916504], [0.11237630248069763], [0.35607174038887024], [0.35607174038887024], [0.092805415391922], [0.0909547209739685], [0.06843438744544983], [0.284619003534317], [0.06897726655006409], [0.9309492707252502], [0.9819850921630

In [0]:
for newdataset in combine:
   
    newdataset['test_survived'] = pd.Series(Y_test)
    newdataset['test_survived'] = newdataset['test_survived'].str.get(0)
    print(newdataset)

In [0]:
series = []
for val in newdataset.test_survived:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
newdataset['final'] = series

In [69]:
test_df.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class,test_survived,final
0,892,3,0,2,0,1,1.0,1,6,0.075821,0
1,893,3,1,2,0,2,3.0,0,6,0.215584,0
2,894,2,0,3,1,1,1.0,1,6,0.122581,0
3,895,3,0,1,1,2,1.0,1,3,0.092805,0
4,896,3,1,1,1,2,3.0,0,3,0.369596,0


In [85]:
test_df.loc[:, ['PassengerId', 'final']]

,PassengerId,final
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
